In [1]:
!pip install tensorflow opencv-python mediapipe scikit-learn matplotlib numpy pandas

In [1]:
# Open CV
import cv2
# For visualizing patterns (for iamshow function_)
from matplotlib import pyplot as plt
# For manipulationof NP-Arrays
import numpy as np
# easy to work with plots
import os
# for sleep function after capturing every frame
import time
# for providing the details of hands, heads,etc in the captured image from the camera
import mediapipe as mp
import pandas as pd

# Use MP Holistic for defining required patterns

In [2]:
mp_holistic = mp.solutions.holistic # Holistic Model
mp_drawing = mp.solutions.drawing_utils # drawing utilities

In [3]:
def mediapipe_detection(image,model):
    # Camera takes input as BGR format (Blue,Green,Red) so we covert it to RGB format first for processing
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False                    # Image should not be writable now
    result = model.process(image)                   # make predictions
    image.flags.writeable = True                     # Make it back to writable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # Back to BGR
    return image, result                             # return the iamge and the results

In [4]:
def draw_landmarks(image,results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # draw right hand connections

In [5]:
def draw_styled_landmarks(image, results):
    # draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                             mp_drawing.DrawingSpec(color = (100,110,90), thickness = 1, circle_radius = 1),
                             mp_drawing.DrawingSpec(color = (69,120,123), thickness = 1, circle_radius = 1))

    # draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color = (189,114,190), thickness = 2, circle_radius = 4),
                             mp_drawing.DrawingSpec(color = (169,125,13), thickness = 2, circle_radius = 3))

    # draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color = (10,10,0), thickness = 2, circle_radius = 2),
                             mp_drawing.DrawingSpec(color = (6,130,123), thickness = 3, circle_radius = 2))

    # draw right hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color = (100,10,9), thickness = 2, circle_radius = 4),
                             mp_drawing.DrawingSpec(color = (190,180,145), thickness = 2, circle_radius = 4))

In [37]:
# for accessing web-cam
cap = cv2.VideoCapture(0)
# Access mideapipe model
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic:
    while cap.isOpened():
        
        # read feed
        ret, frame = cap.read()
    
        # Make detections
        image, results = mediapipe_detection(frame,holistic)
        print(results)

        # Make land_marks
        draw_styled_landmarks(image,results)
        
        # imshow function to show to the user
        cv2.imshow('OpenCV Feed', image)
    
        # break statement to end the loop 
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    # It will release the web-cam for our use
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

# Finding Keypoints Values

## We need to pass some values if Left hand and right hand arent shown int the frame, so we will pass :
    1. A 63 sized np array for left and right hand.
    2. 63 because there are 21 captured for left and right hand - and they have 3 values each - x,y,z (no vilibility)
    3. Hence 21*3 = 63 size np array should be passed as an error control
    4 similary 1404 for face and 132 for pose

In [19]:
def keypoints_extraction(results):
    face = np.array([[res.x,res.y,res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
    pose = np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
    lh = np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(63)
    rh = np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(63)
    return np.concatenate([pose,face,lh,rh])

In [10]:
# data path for eported numpy array
DATA_PATH = os.path.join('MP_DATA')

# Actions we want to detect
actions = np.array(["Hello","Thankyou","Compassion"])

# No of data we will provide the machine (Video)
no_sequence = 30

# Lenght of each video in frames to be provided
sequence_length = 30

In [8]:
for action in actions:
    for sequence in range(no_sequence):
        try:
            os.makedirs(os.path.join(DATA_PATH,action,str(sequence)))
        except:
            pass

# Collecting Keypoints for train and test phase of model

In [26]:
# for accessing web-cam
cap = cv2.VideoCapture(0)
# Access mideapipe model
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic:
    # Loop for traversing actions
    for action in actions:
        # Loop for sequences/videos
        for sequence in range(no_sequence):
            # Loop through video length/sequnece length
            for frame_num in range(sequence_length):
        
                # read feed
                ret, frame = cap.read()
            
                # Make detections
                image, results = mediapipe_detection(frame,holistic)
                print(results)
        
                # Make land_marks
                draw_styled_landmarks(image,results)

                # Applying wait logic to know when it is starting to take input
                if frame_num == 0 :
                    cv2.putText(image,'STARTING SCANNING',(120,200),
                               cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0), 1, cv2.LINE_AA)
                    cv2.putText(image,'SCANNING FRAMES FOR {} VIDEO NUMBER {}'.format(action,sequence),(15,12),
                               cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed',image)
                    cv2.waitKey(2000)
                else:
                    cv2.putText(image,'SCANNING FRAMES FOR {} VIDEO NUMBER {}'.format(action,sequence),(15,12),
                               cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255), 1, cv2.LINE_AA)
                    # show to screen
                    cv2.imshow('OpenCV Feed', image)

                # keypoint extraction
                keypoints = keypoints_extraction(results)
                npy_path = os.path.join(DATA_PATH,action,str(sequence),str(frame_num))
                np.save(npy_path,keypoints)
                
    
                # break statement to end the loop 
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
    # It will release the web-cam for our use
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

# Now will process the data

In [27]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [28]:
label_map = {label: num for num, label in enumerate(actions)}

In [29]:
label_map

{'Hello': 0, 'Thankyou': 1, 'Compassion': 2}

In [35]:
# getting our data
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequence):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH,action,str(sequence),"{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [36]:
np.array(sequences).shape

(90, 30, 1662)

In [37]:
np.array(labels).shape

(90,)

In [38]:
X = np.array(sequences)
X

array([[[ 0.57858145,  0.49208152, -1.47454488, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.55384141,  0.50697815, -1.17251539, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.54412591,  0.51334226, -1.27365994, ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.59529603,  0.50366008, -1.34349847, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.59496617,  0.50221485, -1.37433362, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.59470689,  0.50133741, -1.38178086, ...,  0.        ,
          0.        ,  0.        ]],

       [[ 0.59449559,  0.50110298, -1.34200168, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.60169637,  0.50584322, -1.06508648, ...,  0.295068  ,
          0.51303983, -0.02277706],
        [ 0.60228133,  0.50582951, -1.11051345, ...,  0.28965867,
          0.52662873, -0.00691255],
        ...,
        [ 0.64273942,  0.51431471, -0.70067942, ...,  

In [39]:
Y = to_categorical(labels).astype(int)
Y

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0,

In [41]:
# Split data for test and train
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.05)

# Building and Training LSTM Model

In [45]:
from tensorflow.keras.models import Sequential # make sequential nueral network
from tensorflow.keras.layers import LSTM,Dense
from tensorflow.keras.callbacks import TensorBoard # monitering nueral network train

In [46]:
log_dir = os.path.join('logs')
tb_callback = TensorBoard(log_dir = log_dir)

In [71]:
# Making Nueral Network
model = Sequential()
model.add(LSTM(64, return_sequences = True, activation = 'relu', input_shape = (30,1662)))
model.add(LSTM(128, return_sequences = True, activation = 'relu'))
model.add(LSTM(64, return_sequences = False, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(actions.shape[0], activation = 'softmax'))

In [50]:
model.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = ['categorical_accuracy'])

In [57]:
model.fit(x_train,y_train, epochs = 2000, callbacks= [tb_callback])

Epoch 1/2000
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - categorical_accuracy: 0.8610 - loss: 0.3247
Epoch 2/2000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - categorical_accuracy: 0.8923 - loss: 0.2936
Epoch 3/2000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - categorical_accuracy: 0.8786 - loss: 0.2900
Epoch 4/2000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - categorical_accuracy: 0.8747 - loss: 0.2908
Epoch 5/2000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - categorical_accuracy: 0.8396 - loss: 0.3600
Epoch 6/2000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - categorical_accuracy: 0.8845 - loss: 0.3190
Epoch 7/2000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - categorical_accuracy: 0.8493 - loss: 0.3162
Epoch 8/2000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - categorical_accuracy: 0.8493 - loss: 0.3343
Epoch 9/2000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - categorical_accuracy: 0.8513 - loss: 0.3256
Epoch 10/2000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - categorical_accuracy: 0.8806 - loss: 0.2906
Epoch 11/2000
3/3 ━━━━━━━━━━━


KeyboardInterrupt



In [59]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_6 (LSTM)                        │ (None, 30, 64)              │         442,112 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_7 (LSTM)                        │ (None, 30, 128)             │          98,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_8 (LSTM)                        │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 3)                   │             195 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,784,075 (6.81 MB)

 Trainable params: 594,691 (2.27 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,189,384 (4.54 MB)

# Predict data

In [60]:
model.predict(x_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 812ms/step


array([[9.7010201e-01, 2.0681713e-02, 9.2162183e-03],
       [5.4171352e-16, 6.4070832e-20, 1.0000000e+00],
       [9.8670793e-01, 1.3292055e-02, 2.6699864e-12],
       [1.0000000e+00, 9.1089989e-38, 0.0000000e+00],
       [1.0000000e+00, 9.5115871e-10, 1.6827659e-16]], dtype=float32)

In [61]:
actions[np.argmax(res[0])]

'Hello'

In [62]:
actions[np.argmax(y_test[0])]

'Hello'

In [67]:
model.save('signdetection.h5')

In [68]:
del model

In [72]:
model.load_weights('signdetection.h5')

# Evaluation

In [74]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [75]:
yhat = model.predict(x_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 510ms/step


In [76]:
ytrue = np.argmax(y_test, axis = 1).tolist()
yhat = np.argmax(yhat, axis = 1).tolist()

In [77]:
ytrue

[0, 2, 0, 0, 1]

In [78]:
yhat

[0, 2, 0, 0, 0]

In [79]:
multilabel_confusion_matrix(ytrue,yhat)

array([[[1, 1],
        [0, 3]],

       [[4, 0],
        [1, 0]],

       [[4, 0],
        [0, 1]]], dtype=int64)

In [80]:
accuracy_score(ytrue,yhat)

0.8

In [ ]:
# 1. New variables declaration
sequence =[]
sentence = []
threshold = 0.4

# for accessing web-cam
cap = cv2.VideoCapture(0)
# Access mideapipe model
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic:
    while cap.isOpened():
        
        # read feed
        ret, frame = cap.read()
    
        # Make detections
        image, results = mediapipe_detection(frame,holistic)
        print(results)

        # Make land_marks
        draw_styled_landmarks(image,results)

        # 2. Prediction of handsigns
        keypoints = keypoints_extraction(results)
        sequence.insert(0,keypoints)
        sequence = sequence[:30]

        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis = 0))[0]
            print(actions[np.argmax(res)])

        # 3. Sentence 
            if res[np.argmax(res)] > threshold:
                if len(sentence) > 0 :
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])
            
            if len(sentence) > 5:
                sentence = sentence[-5:]

        # Display sentence
        cv2.rectangle(image, (0,0), (640,40), (255,10,10), -1)
        cv2.putText(image,' '.join(sentence), (3,30),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
            
        # imshow function to show to the user
        cv2.imshow('OpenCV Feed', image)
    
        # break statement to end the loop 
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    # It will release the web-cam for our use
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [85]:
cap.release()
cv2.destroyAllWindows()